### Package Version
- tensorflow==2.2.0

# Sentiment Classification

### Dataset
- Dataset of 50,000 movie reviews from IMDB, labeled by sentiment positive (1) or negative (0)
- Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers).
- For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".
- As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.

Command to import data
- `from tensorflow.keras.datasets import imdb`

### Import the data (2 Marks)
- Use `imdb.load_data()` method
- Get train and test set
- Take 10000 most frequent words

In [ ]:
from tensorflow.keras.datasets import imdb

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = 10000)

### Pad each sentence to be of same length (2 Marks)
- Take maximum sequence length as 300

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(X_train, maxlen = 300)
X_test =  pad_sequences(X_test, maxlen = 300)

### Print shape of features & labels (2 Marks)

Number of review, number of words in each review

In [ ]:
X_train.shape

(25000, 300)

In [ ]:
X_test.shape

(25000, 300)

Number of labels

In [ ]:
y_train.shape

(25000,)

In [ ]:
y_test.shape

(25000,)

### Print value of any one feature and it's label (2 Marks)

Feature value

In [ ]:
X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

Label value

In [ ]:
y_train[1]

0

### Decode the feature value to get original sentence (2 Marks)

First, retrieve a dictionary that contains mapping of words to their index in the IMDB dataset

In [ ]:
word_index_dict = imdb.get_word_index()

Now use the dictionary to get the original words from the encodings, for a particular sentence

In [ ]:
for encoding in X_train[0]:
  for key, value in word_index_dict.items():
    if encoding == value:
      print(key, end = " ")

the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but and to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other and in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of and odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but when from one bit then have tw

Get the sentiment for the above sentence
- positive (1)
- negative (0)

In [ ]:
y_train[0]

1

### Define model (10 Marks)
- Define a Sequential Model
- Add Embedding layer
  - Embedding layer turns positive integers into dense vectors of fixed size
  - `tensorflow.keras` embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unique integer number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn LabelEncoder.
  - Size of the vocabulary will be 10000
  - Give dimension of the dense embedding as 100
  - Length of input sequences should be 300
- Add LSTM layer
  - Pass value in `return_sequences` as True
- Add a `TimeDistributed` layer with 100 Dense neurons
- Add Flatten layer
- Add Dense layer

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Flatten, Dense, TimeDistributed

model = Sequential()
model.add(Embedding(input_dim = 10000, output_dim = 100, input_length = 300))
model.add(LSTM(64, activation = 'relu', return_sequences=True))

model.add(TimeDistributed(Dense(100)))

model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

### Compile the model (2 Marks)
- Use Optimizer as Adam
- Use Binary Crossentropy as loss
- Use Accuracy as metrics

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Print model summary (2 Marks)

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 300, 100)          1000000   
_________________________________________________________________
lstm_6 (LSTM)                (None, 300, 64)           42240     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 300, 100)          6500      
_________________________________________________________________
flatten_5 (Flatten)          (None, 30000)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 1024)              30721024  
_________________________________________________________________
dense_17 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_18 (Dense)             (None, 256)              

### Fit the model (2 Marks)

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=128)

Epoch 1/5
196/196 [==============================] - 149s 763ms/step - loss: 0.4086 - accuracy: 0.7884 - val_loss: 0.2814 - val_accuracy: 0.8836
Epoch 2/5
196/196 [==============================] - 148s 754ms/step - loss: 0.1853 - accuracy: 0.9303 - val_loss: 0.3153 - val_accuracy: 0.8780
Epoch 3/5
196/196 [==============================] - 150s 767ms/step - loss: 0.1020 - accuracy: 0.9626 - val_loss: 0.3947 - val_accuracy: 0.8674
Epoch 4/5
196/196 [==============================] - 152s 776ms/step - loss: 0.0390 - accuracy: 0.9864 - val_loss: 0.6426 - val_accuracy: 0.8615
Epoch 5/5
196/196 [==============================] - 150s 766ms/step - loss: 0.0205 - accuracy: 0.9926 - val_loss: 0.9504 - val_accuracy: 0.8496


### Evaluate model (2 Marks)

In [ ]:
score = model.evaluate(X_test, y_test)
print("Loss: {}, Accuracy:{}".format(score[0], score[1]))

782/782 [==============================] - 39s 50ms/step - loss: 0.9504 - accuracy: 0.8496
Loss: 0.9504016637802124, Accuracy:0.849560022354126


### Predict on one sample (2 Marks)

In [ ]:
model.predict(X_test[0].reshape((1, 300)))

array([[0.00081356]], dtype=float32)